# Evaluating Weather Differences Across Latitudes Using Cities Worldwide

### WeatherAPI data using Python, pandas, matplotlib/seaborn, citipy

-------

#### Python script to visualize the weather of 500+ cities across the world of varying distance from the equator (citipy)

1. Setup

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint

# Import API key
import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


2. gather city data

In [2]:
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        #print(city)
    #print to file        
    with open('city_output.txt', 'w') as f:
                f.write('\n'.join(cities))
# Print the city count to confirm sufficient count
print(len(cities))



632


3. prep and run the api call

In [3]:
from api_keys import api_key

In [4]:


#create df to hold it, start with dict then convert to df

city_weather = {'City':[],
                'Country':[],
                'Cloudiness':[],
                'Date':[],
                'Humidity':[],
                'Lat':[],
                'Lng':[],
                'Max Temp':[],
                'Min Temp':[],
                'Pressure':[],
                'Wind Speed':[],
                'Real Feel Temp':[],
                'City ID':[]}

#loop over cities and add data
for city in cities:
    base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key

    try:
        req=requests.get(base_url + f'&q={city}') 
        data = req.json()
        
        #add values to our city_weather dictionary
        city_weather['City'].append(data['name'])
        city_weather['Country'].append(data['sys']['country'])
        city_weather['Cloudiness'].append(data['clouds']['all'])
        city_weather['Date'].append(data['dt'])
        city_weather['Humidity'].append(data['main']['humidity'])
        city_weather['Lat'].append(data['coord']['lat'])
        city_weather['Lng'].append(data['coord']['lon'])
        city_weather['Max Temp'].append(data['main']['temp_max'])
        city_weather['Min Temp'].append(data['main']['temp_min'])
        city_weather['Pressure'].append(data['main']['pressure'])
        city_weather['Wind Speed'].append(data['wind']['speed'])
        city_weather['Real Feel Temp'].append(data['main']['feels_like'])
        #city_weather['Elevation'].append(data['main']['pressure']) try this with the forecast
        city_weather['City ID'].append(data['id'])
        
        print("processing " + city)
        
    except:
        print(city + " not found")
        pass  #continue on to the next part of loop
    
    #print to file  - not working yet, prints only column headers      
    #with open('city_data.txt', 'w') as f:
                #f.write('\n'.join(city_weather))

processing roebourne
taolanaro not found
processing port hardy
processing bubaque
processing guerrero negro
processing waipawa
processing pandan
processing saint-francois
processing hermanus
processing mataura
processing cidreira
processing arumbavur
processing parauna
processing yellowknife
processing fortuna
processing curup
processing saskylakh
processing jamestown
processing qaanaaq
processing baracoa
processing busselton
processing husavik
processing ponta do sol
tumannyy not found
processing saint-philippe
processing rikitea
processing bredasdorp
processing tiznit
processing sturgeon bay
processing bandarbeyla
processing zhigansk
processing ngama
processing atuona
processing cabedelo
processing punta arenas
processing oga
processing bluff
processing barrow
processing butaritari
processing mar del plata
sentyabrskiy not found
tsihombe not found
processing vostok
processing kodiak
processing khatanga
processing flinders
processing airai
processing oum hadjer
processing oranjestad
p

processing qaqortoq
processing acapulco
processing hohhot
processing mumford
processing le pradet
processing coihaique
processing mega
processing chalus
processing kefamenanu
processing grindavik
processing mogadishu
processing bella vista
processing necochea
processing manokwari
processing cartagena del chaira
processing buturlinovka
processing porto walter
processing itarema
processing amapa
processing yumen
processing onalaska
processing brazzaville
processing wanning
processing dingle
processing rudnichnyy
processing marystown
processing naples
processing pangnirtung
processing bonavista
processing kitui
processing vila franca do campo
processing mackay
processing tonj
processing maumere
adalsbruk not found
processing moron
processing eyl
processing forestville
processing rawah
processing boa vista
processing stephenville crossing
processing coahuayana
processing mosquera
samusu not found
processing pandharpur
processing woodward
processing karatau
processing nurota
processing jiji

In [6]:
print(city_weather)

{'City': ['Roebourne', 'Port Hardy', 'Bubaque', 'Guerrero Negro', 'Waipawa', 'Pandan', 'Saint-François', 'Hermanus', 'Mataura', 'Cidreira', 'Arumbāvūr', 'Paraúna', 'Yellowknife', 'Fortuna', 'Curup', 'Saskylakh', 'Jamestown', 'Qaanaaq', 'Baracoa', 'Busselton', 'Husavik', 'Ponta do Sol', 'Saint-Philippe', 'Rikitea', 'Bredasdorp', 'Tiznit Province', 'Sturgeon Bay', 'Bandarbeyla', 'Zhigansk', 'Ngama', 'Atuona', 'Cabedelo', 'Punta Arenas', 'Oga', 'Bluff', 'Barrow', 'Butaritari', 'Mar del Plata', 'Vostok', 'Saint Paul Harbor', 'Khatanga', 'Flinders', 'Airai', 'Umm Hājar', 'Oranjestad', 'Ushuaia', 'Dalbandin', 'Moindou', 'Hong Kong', 'Hobart', 'East London', 'Agra', 'Keokuk', 'Poum', 'Cape Town', 'Umeå', 'Rawson', 'Beringovskiy', 'Castro', 'Altay', 'Leshukonskoye', 'Mwanza', 'Kashgar', 'Upernavik', 'Lompoc', 'Domna', 'Mongomo', 'Adrar', 'Fernández', 'Hithadhoo', 'Nikolskoye', 'Philadelphia', 'New Norfolk', 'Chokurdakh', 'Vaini', 'Selçuk', 'San Patricio', 'Nizhneudinsk', 'Kavieng', 'Albany', '

 4. get output into pandas dataframe

In [7]:
city_weather_df = pd.DataFrame.from_dict(city_weather)
city_weather_df

,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Min Temp,Pressure,Wind Speed,Real Feel Temp,City ID
0,Roebourne,AU,7,1634333736,67,-20.7833,117.1333,72.63,72.63,1011,13.87,72.73,2062276
1,Port Hardy,CA,75,1634333463,82,50.6996,-127.4199,55.58,51.80,1013,10.36,54.70,6111862
2,Bubaque,GW,2,1634333036,72,11.2833,-15.8333,84.07,84.07,1011,9.80,91.29,2374583
3,Guerrero Negro,MX,2,1634333738,25,27.9769,-114.0611,83.88,83.88,1012,11.63,81.48,4021858
4,Waipawa,NZ,4,1634333739,53,-41.4122,175.5153,57.45,53.92,1017,3.69,53.89,2206874
5,Pandan,PH,68,1634333740,84,14.0453,124.1694,77.81,77.81,1008,2.59,79.23,1695546
6,Saint-François,FR,0,1634333741,95,46.4154,3.9054,41.68,39.47,1020,0.00,40.91,2980080
7,Hermanus,ZA,48,1634333282,70,-34.4187,19.2345,53.91,53.80,1017,19.75,52.16,3366880
8,Mataura,NZ,100,1634333232,60,-46.1927,168.8643,50.22,50.22,1015,4.03,47.75,6201424
9,Cidreira,BR,100,1634333487,92,-30.1811,-50.2056,66.18,66.18,1012,18.01,66.83,3466165


 5. convert unix time to date-time

In [ ]:
#converting unix time to date-time


from datetime import datetime
ts = int('1284101485')

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

 6. create scatterplots using matplotlib showcasing the following relationships:

* Temperature (F) vs. Latitude
* Humidity (%) vs. Latitude
* Cloudiness (%) vs. Latitude
* Wind Speed (mph) vs. Latitude

In [ ]:
# create individual scatterplots, then do subplots

# Import the matplotlib.pyplot submodule and name it plt
import matplotlib.pyplot as plt

# Create a Figure and an Axes with plt.subplots
fig, ax = plt.subplots()


# Plot Seattle data, setting data appearance
ax.plot(seattle_weather["MONTH"], seattle_weather["MLY-PRCP-NORMAL"], marker="o", linestyle= "--", color="b")

# Plot Austin data, setting data appearance
ax.plot(austin_weather["MONTH"], austin_weather["MLY-PRCP-NORMAL"], marker="v",linestyle="--",color="r")

# Call show to display the resulting plot
plt.show()

 7. create scatterplots using seaborn showcasing the following relationships:

* Temperature (F) vs. Latitude
* Humidity (%) vs. Latitude
* Cloudiness (%) vs. Latitude
* Wind Speed (mph) vs. Latitude

## three observable trends based on the data

Your final notebook must:

* Randomly select **at least** 500 unique (non-repeat) cities based on latitude and longitude.
* Perform a weather check on each of the cities using a series of successive API calls.
* OPTIONAL: Include a print log of each city as it's being processed with the city number and city name.
* OPTIONAL: Save both a CSV of all data retrieved and png images for each scatter plot.

## Note that the cities tend to be the same when you re-run it with citipy - investigate how "random" it really is.  Bootstrap it?

## OPTIONAL: Include a print log of each city as it's being processed with the city number and city name.

## OPTIONAL: Save both a CSV of all data retrieved and png images for each scatter plot.

## Hints and Considerations

* You may want to start this assignment by refreshing yourself on the [geographic coordinate system](http://desktop.arcgis.com/en/arcmap/10.3/guide-books/map-projections/about-geographic-coordinate-systems.htm).

* Next, spend the requisite time necessary to study the OpenWeatherMap API. Based on your initial study, you should be able to answer  basic questions about the API: Where do you request the API key? Which Weather API in particular will you need? What URL endpoints does it expect? What JSON structure does it respond with? Before you write a line of code, you should be aiming to have a crystal clear understanding of your intended outcome.

 